使用RandomOverSampler进行过采样，过拟合现象比较严重

In [33]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns
import imblearn
from plotnine import *

In [34]:
raw_structure_data = pd.read_csv('csv_building_structure.csv')

In [35]:
raw_structure_data.dropna(inplace=True)
raw_structure_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 762094 entries, 0 to 762105
Data columns (total 31 columns):
 #   Column                                  Non-Null Count   Dtype 
---  ------                                  --------------   ----- 
 0   building_id                             762094 non-null  int64 
 1   district_id                             762094 non-null  int64 
 2   vdcmun_id                               762094 non-null  int64 
 3   ward_id                                 762094 non-null  int64 
 4   count_floors_pre_eq                     762094 non-null  int64 
 5   count_floors_post_eq                    762094 non-null  int64 
 6   age_building                            762094 non-null  int64 
 7   plinth_area_sq_ft                       762094 non-null  int64 
 8   height_ft_pre_eq                        762094 non-null  int64 
 9   height_ft_post_eq                       762094 non-null  int64 
 10  land_surface_condition                  762094 non-null 

In [17]:
distrct_map = pd.read_csv("ward_vdcmun_district_name_mapping.csv")
structure_data = pd.merge(raw_structure_data, distrct_map, on=['ward_id', 'vdcmun_id', 'district_id'], how='left')

In [18]:
structure_data = structure_data.drop(['building_id', 'district_id', 'vdcmun_id', 'ward_id', 'count_floors_post_eq', \
    'height_ft_post_eq', 'condition_post_eq', 'technical_solution_proposed', 'vdcmun_name'], axis=1)

In [19]:
structure_data['damage_grade'].value_counts()

Grade 5    275766
Grade 4    183844
Grade 3    136412
Grade 2     87257
Grade 1     78815
Name: damage_grade, dtype: int64

In [20]:
print(structure_data.foundation_type.value_counts())

Mud mortar-Stone/Brick    628705
Bamboo/Timber              57472
Cement-Stone/Brick         39245
RC                         32120
Other                       4552
Name: foundation_type, dtype: int64


In [21]:
structure_data['foundation_type'] = structure_data['foundation_type'].replace(['Bamboo/Timber', 'Cement-Stone/Brick', 'RC'], 'Other')


In [22]:
numeric_variables = structure_data.select_dtypes(include=['float64', 'int64'])

log_transformed_data = np.log1p(numeric_variables)

structure_data[numeric_variables.columns] = log_transformed_data

In [23]:
severity_mapping = {'Grade 1': 1, 'Grade 2': 2, 'Grade 3': 3, 'Grade 4': 4, 'Grade 5': 5}
structure_data['damage_grade'] = structure_data['damage_grade'].map(severity_mapping)

In [24]:
columns_to_encode = ['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type',
                      'other_floor_type', 'position', 'plan_configuration']

structure_data = pd.get_dummies(structure_data, columns=columns_to_encode)

In [25]:
structure_data['district_name'].value_counts()

Kavrepalanchok    98019
Makwanpur         90994
Dhading           89122
Sindhupalchok     88741
Gorkha            78074
Nuwakot           77148
Sindhuli          68749
Dolakha           60639
Ramechhap         58612
Okhaldhunga       39352
Rasuwa            12644
Name: district_name, dtype: int64

In [26]:
grouped_dataframes = structure_data.groupby('district_name')
district_dataframes = {name: group for name, group in grouped_dataframes}

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

scaler = StandardScaler()

for name, group in grouped_dataframes:
    X = group.drop(['district_name', 'damage_grade'], axis=1)
    y = group['damage_grade']
    
    # 使用RandomOverSampler进行过采样
    oversampler = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = oversampler.fit_resample(X, y)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    rf_classifier = RandomForestClassifier(random_state=42, n_estimators=1000, max_depth=3, n_jobs=-1)
    rf_classifier.fit(X_train_scaled, y_train)

    y_pred = rf_classifier.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy of {name}: {accuracy:.2f}')

exception calling callback for <Future at 0x25710043e50 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "D:\anaconda1\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 359, in __call__
    **backend_params
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 794, in dispatch_next
    # registers the task outcome (TASK_ERROR or TASK_DONE), and schedules
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    with self.parallel._lock:
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
  File "D:\anaconda1\lib\site-packages\joblib\_parallel_backends.py", line 531, in apply_async
    """Shutdown the process or thread pool"""
  File "D:\anaconda1\lib\site-packages\joblib\externals\loky\reusable_executor.py", line 177, in submit
    mp.util.debug(
  File "D:\anaconda1\lib\sit

Cross-validation scores for Dhading: [nan nan nan nan nan]
Mean cross-validation accuracy for Dhading: nan
Accuracy of Dhading: 0.40


exception calling callback for <Future at 0x25703d7fa00 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "D:\anaconda1\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 359, in __call__
    **backend_params
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 794, in dispatch_next
    # registers the task outcome (TASK_ERROR or TASK_DONE), and schedules
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    with self.parallel._lock:
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
  File "D:\anaconda1\lib\site-packages\joblib\_parallel_backends.py", line 531, in apply_async
    """Shutdown the process or thread pool"""
  File "D:\anaconda1\lib\site-packages\joblib\externals\loky\reusable_executor.py", line 177, in submit
    mp.util.debug(
  File "D:\anaconda1\lib\sit

Cross-validation scores for Dolakha: [nan nan nan nan nan]
Mean cross-validation accuracy for Dolakha: nan
Accuracy of Dolakha: 0.39


exception calling callback for <Future at 0x2570c53ceb0 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "D:\anaconda1\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 359, in __call__
    **backend_params
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 794, in dispatch_next
    # registers the task outcome (TASK_ERROR or TASK_DONE), and schedules
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    with self.parallel._lock:
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
  File "D:\anaconda1\lib\site-packages\joblib\_parallel_backends.py", line 531, in apply_async
    """Shutdown the process or thread pool"""
  File "D:\anaconda1\lib\site-packages\joblib\externals\loky\reusable_executor.py", line 177, in submit
    mp.util.debug(
  File "D:\anaconda1\lib\sit

...

Cross-validation scores for Okhaldhunga: [nan nan nan nan nan]
Mean cross-validation accuracy for Okhaldhunga: nan
Accuracy of Okhaldhunga: 0.32


exception calling callback for <Future at 0x257035c1940 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "D:\anaconda1\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 359, in __call__
    **backend_params
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 794, in dispatch_next
    # registers the task outcome (TASK_ERROR or TASK_DONE), and schedules
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    with self.parallel._lock:
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
  File "D:\anaconda1\lib\site-packages\joblib\_parallel_backends.py", line 531, in apply_async
    """Shutdown the process or thread pool"""
  File "D:\anaconda1\lib\site-packages\joblib\externals\loky\reusable_executor.py", line 177, in submit
    mp.util.debug(
  File "D:\anaconda1\lib\sit

Cross-validation scores for Ramechhap: [nan nan nan nan nan]
Mean cross-validation accuracy for Ramechhap: nan
Accuracy of Ramechhap: 0.36


exception calling callback for <Future at 0x257035d9b80 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "D:\anaconda1\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 359, in __call__
    **backend_params
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 794, in dispatch_next
    # registers the task outcome (TASK_ERROR or TASK_DONE), and schedules
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    with self.parallel._lock:
  File "D:\anaconda1\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
  File "D:\anaconda1\lib\site-packages\joblib\_parallel_backends.py", line 531, in apply_async
    """Shutdown the process or thread pool"""
  File "D:\anaconda1\lib\site-packages\joblib\externals\loky\reusable_executor.py", line 177, in submit
    mp.util.debug(
  File "D:\anaconda1\lib\sit

Cross-validation scores for Rasuwa: [nan nan nan nan nan]
Mean cross-validation accuracy for Rasuwa: nan
Accuracy of Rasuwa: 0.52
